In [39]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

In [40]:
df_casual = pd.read_pickle('../../datos_finales/archivo_casual_est_enc2.pkl')
df_casual.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,usuarios_casuales
0,1.0,1.0,1.0,2.67,1.79,4.25,-0.486274,0.855582,-0.208909,331
1,1.0,1.0,1.0,1.83,1.00,4.25,-0.425690,0.332722,0.687550,131


In [41]:
lista_cols = ['estacion', 'velocidad_viento',
       'humedad', 'año', 'dias_laborables', 'tiempo']

In [42]:
df_casual.drop(lista_cols, axis = 1, inplace = True)

In [43]:
X = df_casual.drop("usuarios_casuales", axis = 1)
y = df_casual["usuarios_casuales"]

In [44]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [45]:
param = {"max_depth": [4,6,8], 
        "max_features": [1,2,3],
        'subsample'     : [0.5, 1], # parametros que se usan de forma estandar
        'learning_rate' : [0.001, 0.01, 0.1]} # parametros que se usan de forma estandar

In [46]:
gs_gb = GridSearchCV(
            estimator=GradientBoostingRegressor(), 
            param_grid= param, 
            cv=10, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [47]:
gs_gb.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [4, 6, 8], 'max_features': [1, 2, 3],
                         'subsample': [0.5, 1]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [48]:
boosting = gs_gb.best_estimator_
boosting

GradientBoostingRegressor(max_depth=4, max_features=1, subsample=0.5)

In [49]:
y_pred_test_gb = boosting.predict(x_test)
y_pred_train_gb = boosting.predict(x_train)

In [55]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [56]:
dt_results = metricas(y_test, y_train, y_pred_test_gb, y_pred_train_gb, "Gradient Boosting")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,338.232052,250650.828046,500.650405,0.478272,test,Gradient Boosting
1,228.662273,105890.066337,325.407539,0.773597,train,Gradient Boosting


In [57]:

importancia_predictores = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': boosting.feature_importances_}
                            )


importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
2,temperatura,0.449842
0,mes,0.317624
1,dia_semana,0.232533
